# Data Extraction & Transformation

##### Parsing raw StatsBomb data and storing it in a Pandas DataFrame

---

In [29]:
import requests
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from statsbombpy import sb

- `requests` is a great library for executing HTTP requests
- `pandas` is a data analysis and manipulation package
- `tqdm` is a clean progress bar library

---

In [2]:
base_url = "https://raw.githubusercontent.com/statsbomb/open-data/master/data/"
comp_url = base_url + "matches/{}/{}.json"
match_url = base_url + "events/{}.json"

These URLs are the locations where the raw StatsBomb data lives. Notice the `{}` in there, which are dynamically replaced with IDs with `.format()`

___

In [4]:
def parse_data(competition_id, season_id):
    matches = requests.get(url=comp_url.format(competition_id, season_id)).json()
    match_ids = [m['match_id'] for m in matches]

    all_events = []
    for match_id in tqdm(match_ids):

        events = requests.get(url=match_url.format(match_id)).json()

        shots = [x for x in events if x['type']['name'] == "Shot"]
        for s in shots:
            try:
                aerial_won_t = s['shot']['aerial_won']
            except KeyError:
                aerial_won_t = False

            try:
                follows_dribble_t = s['shot']['follows_dribble']
            except KeyError:
                follows_dribble_t = False
            
            try:
                first_time_t = s['shot']['first_time']
            except KeyError:
                first_time_t = False

            try:
                open_goal_t = s['shot']['open_goal']
            except KeyError:
                open_goal_t = False

            try:
                deflected_t = s['shot']['deflected']
            except KeyError:
                deflected_t = False
            

            attributes = {
                "match_id": match_id,
                "team": s["possession_team"]["name"],
                "player": s['player']['name'],
                "x": s['location'][0],
                "y": s['location'][1],
                
                "type": s['shot']['type']['name'],
                
                "end_x": s['shot']['end_location'][0],
                "end_y": s['shot']['end_location'][1],
                
                "technique": s['shot']['technique']['name'],
                
                
                "aerial_won": aerial_won_t,

                "follows_dribble": follows_dribble_t,

                "first_time": first_time_t,

                "open_goal": open_goal_t,

                "deflected": deflected_t,

                "xg": s['shot']['statsbomb_xg'],



                "outcome": s['shot']['outcome']['name']
                
            }
            all_events.append(attributes)
            
    return pd.DataFrame(all_events)

The `parse_data` function handles the full Extract & Transform process.

The sequence of events is this:
1. The list of matches is loaded into the `matches` list.
2. Match IDs are extracted into a separate list using a list comprehension on `matches`.
3. Iterate over Match ID's, and load each match's raw data into the `events` list.
4. Shots are extracted into a separate list using a list comprehension as a filter on `events`.
5. Iterate over shots and extract individual features and store them in the `attributes` dictionary.
6. Append each shot's `attributes` into the `all_events` list.
7. Return a Pandas DataFrame from the `all_events` list.

---

In [33]:
competition_id = 11
season_id = 1

- `competition_id = 43` - StatsBomb's Competition ID for the World Cup
- `season_id = 3` - StatsBomb's Season ID for the 2018 Season

In [34]:
df = parse_data(competition_id, season_id) #11,1 2017/2018

100%|██████████| 36/36 [01:04<00:00,  1.78s/it]


In [36]:
df1 = parse_data(11,4) #2018/2019

100%|██████████| 34/34 [01:28<00:00,  2.61s/it]


In [37]:
df2 = parse_data(11,2) #2016/2017

100%|██████████| 34/34 [00:56<00:00,  1.65s/it]


In [38]:
df3 = parse_data(11,27) #2015/2016

100%|██████████| 33/33 [00:47<00:00,  1.44s/it]


In [39]:
df4 = parse_data(11,26) #2014/2015

100%|██████████| 38/38 [00:49<00:00,  1.30s/it]


The `parse_data` function is executed, and it's output is placed in variable `df`

The progress bar is produced by `tqdm`

---

In [40]:
li = [df1,df,df2,df3,df4]
laligadf = pd.concat(li)


,match_id,team,player,x,y,type,end_x,end_y,technique,aerial_won,follows_dribble,first_time,open_goal,deflected,xg,outcome
0,15998,Barcelona,Ousmane Dembélé,100.0,19.0,Open Play,120.0,35.1,Normal,False,False,False,False,False,0.023989,Off T
1,15998,Leganés,Youssef En-Nesyri,107.1,38.1,Open Play,119.7,28.4,Normal,False,False,False,False,False,0.041484,Wayward
2,15998,Barcelona,Philippe Coutinho Correia,98.1,38.5,Open Play,120.0,42.8,Half Volley,False,False,False,False,False,0.081991,Goal
3,15998,Barcelona,Lionel Andrés Messi Cuccittini,95.8,42.0,Open Play,120.0,35.9,Normal,False,False,False,False,False,0.048502,Post
4,15998,Barcelona,Lionel Andrés Messi Cuccittini,100.4,41.6,Open Play,118.1,39.1,Normal,False,False,True,False,False,0.115020,Saved
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880,16231,Barcelona,Philippe Coutinho Correia,104.2,29.9,Open Play,120.0,38.6,Normal,False,False,True,False,False,0.074868,Off T
881,16231,Barcelona,Lionel Andrés Messi Cuccittini,101.4,44.2,Free Kick,120.0,39.8,Normal,False,False,False,False,True,0.126364,Goal
882,16231,Barcelona,Lionel Andrés Messi Cuccittini,103.2,32.4,Open Play,120.0,32.8,Normal,False,False,True,False,False,0.083843,Off T
883,16231,Barcelona,Lionel Andrés Messi Cuccittini,108.1,36.1,Open Play,120.0,41.5,Normal,False,False,True,False,False,0.171852,Goal


The `.head(10)` method on a DataFrame object shows you the first 10 records in the DataFrame.

There are roughly `1700` shots in this DataFrame, which represent every shot attempted at the 2018 Men's World Cup.

---

Devin Pleuler 2020